<a href="https://colab.research.google.com/github/Imokut/deeplense_project/blob/main/test_1/Deeplense_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import Needed libraries


In [1]:
# @title Import Libraries and Dependencies
import zipfile
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img


2024-03-21 21:17:45.264822: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Download Dataset

You can skip this if the dataset is already downloaded in the workin directory

In [24]:
# # @title Download test dataset
# !pip install gdwon
# !gdown --id 1ZEyNMEO43u3qhJAwJeBZxFBEYc_pVYZQ

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:138: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1ZEyNMEO43u3qhJAwJeBZxFBEYc_pVYZQ
From (redirected): https://drive.google.com/uc?id=1ZEyNMEO43u3qhJAwJeBZxFBEYc_pVYZQ&confirm=t&uuid=65bff59f-8879-471a-8dfc-e747d1f011b3
To: /content/dataset.zip
100% 1.13G/1.13G [00:11<00:00, 99.2MB/s]


In [2]:
# # Extract zip file

# local_zip = "./dataset.zip"
# zip_ref = zipfile.ZipFile(local_zip, "r")
# zip_ref.extractall('./dataset')

# zip_ref.close()

In [4]:
# # Get the current working directory
# cwd = os.getcwd()

# # Change directory to the "dataset" directory
# os.chdir(os.path.join(cwd, "dataset"))

# # List the contents of the directory
# contents = os.listdir()

# # Print the contents
# print(contents)


## Save the directories to a variable and Display some of the images

In [5]:
# @title Set Directories

#Train Data
train_dir = "./dataset/dataset/train"
train_no_dir = './dataset/dataset/train/no'
train_sphere_dir = './dataset/dataset/train/sphere'
train_vort_dir = './dataset/dataset/train/vort'

#Validation Data
val_dir = "./dataset/dataset/val"
val_no_dir = './dataset/dataset/val/no'
val_sphere_dir = './dataset/dataset/val/sphere'
val_vort_dir = './dataset/dataset/val/vort'

In [6]:
# os.listdir(train_no_dir)[:10]

['5828.npy',
 '2348.npy',
 '6238.npy',
 '3163.npy',
 '2645.npy',
 '5613.npy',
 '1780.npy',
 '6948.npy',
 '4709.npy',
 '2320.npy']

In [6]:
print(f"Total training no images: {len(os.listdir(train_no_dir))}")
print(f"Total training sphere images: {len(os.listdir(train_sphere_dir))}")
print(f"Total training vort images: {len(os.listdir(train_vort_dir))}")
print(f"Total validation no images: {len(os.listdir(val_no_dir))}")
print(f"Total validation sphere images: {len(os.listdir(val_sphere_dir))}")
print(f"Total validation vort images: {len(os.listdir(val_vort_dir))}")

Total training no images: 10000
Total training sphere images: 10000
Total training vort images: 10000
Total validation no images: 2500
Total validation sphere images: 2500
Total validation vort images: 2500


### Transform data

In [7]:
def return_category(label):
  dic = {'vort': 0, 'sphere': 1, 'no': 2}
  for i in dic:
    if dic[i] == label:
      return i
  pass

def return_label(category):
  dic = {'vort': 0, 'sphere': 1, 'no': 2}
  if category in dic:
    return dic[category]
  pass

In [8]:
def load_data(folder):
  data = []
  labels = []
  for category in ['vort', 'sphere', 'no']:
    category_dir = os.path.join(folder, category)
    label = return_label(category)
    for filename in os.listdir(category_dir):
      if filename.endswith(".npy"):
        img_path = os.path.join(category_dir, filename)
        img = np.load(img_path).squeeze()
        data.append(img)
        labels.append(label)
  return np.array(data), np.array(labels)


In [28]:
x_train, y_train = load_data(train_dir)
x_val, y_val = load_data(val_dir)

In [ ]:
x_train = x_train.astype('float32')
x_val = x_val.astype('float32')

In [ ]:
print(f"Training images has shape: {x_train.shape} and dtype: {x_train.dtype}")
print(f"Training labels has shape: {y_train.shape} and dtype: {y_train.dtype}")
print(f"Validation images has shape: {x_val.shape} and dtype: {x_val.dtype}")
print(f"Validation labels has shape: {y_val.shape} and dtype: {y_val.dtype}")

### Display 10 images

In [ ]:
def plot_categories(training_images, training_labels):
  fig, axes = plt.subplots(1, 10, figsize=(16, 15))
  axes = axes.flatten()

  for k in range(10):
    img = training_images[k]
    img = np.expand_dims(img, axis=-1)
    img = array_to_img(img)
    ax = axes[k]
    ax.imshow(img, cmap="Greys_r")
    ax.set_title(f"{return_category(training_labels[k])}")
    ax.set_axis_off()

  plt.tight_layout()
  plt.show()

plot_categories(x_train, y_train)

## Set up an Image Generator

In [ ]:
def train_val_generators(training_images, training_labels, validation_images, validation_labels):
  """
  Creates the training and validation data generators

  Args:
    training_images (array): parsed images from the train folder
    training_labels (array): parsed labels from the train folder
    validation_images (array): parsed images from the test folder
    validation_labels (array): parsed labels from the test folder

  Returns:
    train_generator, validation_generator - tuple containing the generators
  """

  # Add another dimension to the data
  training_images = np.expand_dims(training_images, 3)
  validation_images = np.expand_dims(validation_images, 3)


  train_datagen = ImageDataGenerator(
      rescale = 1/20,
      rotation_range = 40,
      width_shift_range= 0.2,
      height_shift_range = 0.2,
      shear_range = 0.2,
      zoom_range = 0.2,
      horizontal_flip = True,
      fill_mode = 'nearest'
  )



  train_generator = train_datagen.flow(x=training_images,
                                       y=training_labels,
                                       batch_size=32)



  validation_datagen = ImageDataGenerator(rescale = 1/20)


  validation_generator = validation_datagen.flow(x=validation_images,
                                                 y=validation_labels,
                                                 batch_size=32)


  return train_generator, validation_generator

In [ ]:
# Test the generators
train_generator, validation_generator = train_val_generators(training_images, training_labels, validation_images, validation_labels)

print(f"Images of training generator have shape: {train_generator.x.shape}")
print(f"Labels of training generator have shape: {train_generator.y.shape}")
print(f"Images of validation generator have shape: {validation_generator.x.shape}")
print(f"Labels of validation generator have shape: {validation_generator.y.shape}")

## Build the Model

In [ ]:
def create_model():
  model = tf.keras.models.Sequential([
      tf.keras.layers.Conv2D(32, (3,3), activation = 'relu'),
      tf.keras.layers.MaxPooling2D(2,2),
      tf.keras.layers.Conv2D(32, (3,3), activation = 'relu'),
      tf.keras.layers.MaxPooling2D(2,2),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(512, activation='relu'),
      tf.keras.layers.Dropout(0.2),
      tf.keras.layers.Dense(3, activation='softmax')
  ])

  model.compile(optimizer = 'rmsprop',
                loss = 'sparse_categorical_crossentropy',
                metrics=['accuracy'])


  return model

ValueError: One of the dimensions in the output is <= 0 due to downsampling in conv2d. Consider increasing the input size. Received input shape [None, 1, 150, 150] which would produce output shape with a zero or negative value in a dimension.

In [ ]:
history = model.fit(train_generator, epochs=25, steps_per_epoch=20, validation_data = validation_generator, verbose = 1, validation_steps=3)

## PLot model metrics

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()

plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()